In [1]:
import cv2
import mediapipe as mp
import pyautogui
import time

# MediaPipe el algılama ve çizim araçlarını başlat
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# El tanima modeli
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)

# Pozisyon icin
screen_width, screen_height = pyautogui.size()

cap = cv2.VideoCapture(0)
mouse_enabled = False
action_text = "bekleniyor"

last_positions = {"index": 0, "middle": 0, "ring": 0, "pinky": 0}


last_click_time = 0
click_cooldown = 0.5  

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # MediaPipe -> Python -> Solutions : Mediapipe RGB kullaniyor, ayni zamanda flip sebebi kendi dokumantasyonunda verilmistir. Daha net sonuclar icin ayna goruntusu alinmalidir.
    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    # Çizim için görüntü
    frame_height, frame_width, _ = frame.shape
    image = frame.copy()

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Parmak uclari
            tips = {
                "thumb": hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP],
                "index": hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP],
                "middle": hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP],
                "ring": hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_TIP],
                "pinky": hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_TIP],
            }

            # El MCP'ler
            mcps = {
                "index": hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP],
                "middle": hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_MCP],
                "ring": hand_landmarks.landmark[mp_hands.HandLandmark.RING_FINGER_MCP],
                "pinky": hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP],
            }

            # Fare ac-kapat : Basparmak ve isaret parmagi yakin kosulu
            if (
                abs(tips["thumb"].x - tips["index"].x) < 0.05
                and abs(tips["thumb"].y - tips["index"].y) < 0.05
            ):
                mouse_enabled = not mouse_enabled
                action_text = "Fare Kontrol Acildi" if mouse_enabled else "Fare Kontrol Kapandi"
                time.sleep(0.5)  # Tetikleme için kısa bekleme süresi

            # Yumruk ile fare hareketi
            if mouse_enabled:
                is_fist = all(
                    tips[finger].y > mcps[finger].y
                    for finger in ["index", "middle", "ring", "pinky"]
                )
                if is_fist:
                    index_x, index_y = (
                        int(tips["index"].x * screen_width),
                        int(tips["index"].y * screen_height),
                    )
                    pyautogui.moveTo(index_x, index_y)
                    action_text = "Fare Hareket Ediyor (Yumruk)"

            # Tikla : isaret parmagi acik
            elif (
                tips["index"].y < mcps["index"].y
                and all(
                    tips[finger].y > mcps[finger].y
                    for finger in ["middle", "ring", "pinky"]
                )
                and time.time() - last_click_time > click_cooldown
            ):
                pyautogui.click()
                last_click_time = time.time()
                action_text = "Tıklama Yapıldı"

            # Kaydirma
            else:
                delta_y = sum(
                    tips[finger].y - last_positions[finger]
                    for finger in ["index", "middle", "ring", "pinky"]
                )
                if delta_y > 0.02:  # Yukarı kaydır
                    pyautogui.scroll(50)
                    action_text = "Yukari kaydiriliyor"
                elif delta_y < -0.02:  # Aşağı kaydır
                    pyautogui.scroll(-50)
                    action_text = "Asagi kaydiriliyor"

            # Son pozisyonlari guncelle
            for finger in tips:
                last_positions[finger] = tips[finger].y

    # Durum mesaji
    cv2.putText(image, action_text, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Görüntüyü göster
    cv2.imshow("Hands as Mouse", image)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
